Building a Unigram and custom Dataset for training with `T5`
--------------------------------

Like we did in [processing_2](text_processing2.ipynb) to build a tokenizer for GPT-2 we will need to create one for the T5 model. We will train a Unigram Tokenizer for both of the French and Wolof corpora and finally a custom dataset to recuperate the tokenized sentences.

To understand how is working the Unigram tokenizer, see the following tutorial [Unigram_tokenizer](https://huggingface.co/learn/nlp-course/chapter6/7?fw=pt).

The following steps will be necessary to achieve our task:

- Creating a batch generator to load the sentences when training the tokenizer.
- Load the Unigram Tokenizer from the `tokenizers` library.
- Add a normalizer to the tokenizer: See the following link for an explanation of the different types of normalizers [normalizer](https://unicode.org/reports/tr15/). But we will only need to remove too much space that will be found inside the sentences since we have already replaced any weird signs in the corpora (see [extract_sentence](extract_sentences.ipynb) and [extract_text](text_extraction.ipynb)).
- Initialize the pre-tokenizer.
- Initialize the trainer: we will need to furnish the special tokens that will be used and the vocab size. Let us take, for the latter, 10000 tokens for each corpus.
- Train the tokenizer.
- Initialize the post-processor `TemplateProcessing`: we will define the types' ids.
- Initialize the decoder: `Metaspace.`
- Make an example with some sentences.
- Save the tokenizers
- Create the custom dataset for the T5 model.

Let us import the necessary libraries.

In [99]:
# for creating the tokenizer
from tokenizers import (
    decoders,
    models,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
    normalizers
)

# for importing and manipulating the sentences
import pandas as pd
import random

# for loading sentences with the custom dataset
from torch.utils.data import DataLoader

#### Load dataset and create generator

We will create one tokenizer for both of the French and Wolof corpora because the `T5` model' understand only one embedding layer. So we must create one generator for both of the French and Wolof corpora. 

In [108]:
# load sentences
sentences = pd.read_csv("data/extractions/new_data/corpora_v3.csv")

# initialize a batch size
BATCH_SIZE = 120

# create generators (for the corpora)
def generate_sents():
    
    # recuperate the sentences
    french_sents = sentences['french_corpus'].to_list() 
    
    wolof_sents = sentences['wolof_corpus'].to_list() 
    
    sents = french_sents + wolof_sents
    
    for i in range(1, len(sents), BATCH_SIZE):
        
        yield sents[i:i+BATCH_SIZE]

#### Initialize the tokenizer

In [109]:
tokenizer = Tokenizer(models.Unigram())

#### Add normalizer

In [110]:
tokenizer.normalizer = normalizers.Replace(" {2,}", " ")

#### Configure the pre-tokenizers

We will use the Metaspace pre-tokenizer which separates the words considering the spaces between them. It will replace the space by a character (by default the underscore "_").

In [111]:
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

#### Initialize the trainers

We will provide all of the necessary special tokens to the Trainer. 

**Notice that a sentence can be a group of words separated by ending marks and not only one group of words. Then we can, for example, tokenize the following sentences**: `<sep>sentence1.sentence2.sentence3<cls>` **or** `<sep>sentence1.<sep>sentence2.<cls>`. **But, the second sentence is composed of two separate groups. Then the two sentences will have different type ids.** 

In [112]:
special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]

In [113]:
trainer = trainers.UnigramTrainer(vocab_size=10000, special_tokens=special_tokens, unk_token = "<unk>")

#### Train the tokenizer

In [114]:
tokenizer.train_from_iterator(generate_sents(), trainer)

Let us print the vocab size.

In [115]:
print(f"Number of tokens: {tokenizer.get_vocab_size()}")

Number of tokens: 7611


#### Initialize the post-processor

We can not need the TemplateProcessor to train our corpora in a Sequence To Sequence model, but we will add it to our tokenizer. We can use it for another type of model. 

In [116]:
# let us recuperate the sep and cls ids
cls_token_id = tokenizer.token_to_id("<cls>")

sep_token_id = tokenizer.token_to_id("<sep>")

print(cls_token_id, sep_token_id)

0 1


In [117]:
# Initialize the post processor
tokenizer.post_process = processors.TemplateProcessing(
    single="$A:0 <sep>:0 <cls>:2",
    pair="$A:0 <sep>:0 $B:1 <sep>:1 <cls>:2",
    special_tokens=[("<sep>", sep_token_id), ("<cls>", cls_token_id)]
)

#### Initialize the decoder

In [118]:
tokenizer.decoder = decoders.Metaspace()

#### Save the tokenizer

In [119]:
tokenizer.save("wolof-translate/wolof_translate/tokenizers/t5_tokenizers/tokenizer_v1.json")

#### Make a little example

Let us recuperate random sentences from the corpora and tokenize them.

In [120]:
random.seed(100)

french_sentence = random.choice(sentences['french_corpus'])

wolof_sentence = random.choice(sentences['wolof_corpus'])

In [121]:
# print the french sentence
french_sentence

"Nous frappions à nouveau, jusqu'à en avoir mal aux mains, comme si nous combattions un ennemi invisible."

In [122]:
# print the wolof sentence
wolof_sentence

'Mu ngi dale digaloom ak Kamerun gi ci moomeelu Farañse yi, wuti bëj-gànnaar, ca cati Aadamawa, ëmbaale dëkk yu ndaw-ndawaan yi Àngale yi jotul woon a teg loxo ba ñu fa dàqee Almaŋ yi. Muy dëkk yu mel ni : Kantu, Aboŋ, Nkom, Bum, Fumbaŋ ak Baali.'

In [123]:
french_encoding = tokenizer.encode(french_sentence)

print("French tokens")
print(french_encoding.tokens)

print("French ids")
print(french_encoding.ids)

French tokens
['▁N', 'ous', '▁frappions', '▁à', '▁nouveau', ',', "▁jusqu'", 'à', '▁en', '▁', 'avoir', '▁mal', '▁aux', '▁mains', ',', '▁comme', '▁si', '▁nous', '▁combatt', 'ions', '▁un', '▁ennemi', '▁invisible', '.']
French ids
[188, 441, 2158, 20, 925, 7, 329, 105, 40, 8, 738, 707, 206, 957, 7, 137, 127, 110, 3442, 309, 41, 4829, 2648, 9]


In [124]:
wolof_encoding = tokenizer.encode(wolof_sentence)

print("Wolof tokens")
print(wolof_encoding.tokens)

print("Wolof ids")
print(wolof_encoding.ids)

Wolof tokens
['▁Mu', '▁ngi', '▁dal', 'e', '▁digal', 'oom', '▁', 'ak', '▁Kamerun', '▁gi', '▁ci', '▁moom', 'eelu', '▁Far', 'añse', '▁yi', ',', '▁wuti', '▁bëj-gànnaar', ',', '▁ca', '▁c', 'ati', '▁Aadamawa', ',', '▁ëmb', 'aale', '▁dëkk', '▁yu', '▁ndaw', '-', 'ndaw', 'aan', '▁yi', '▁Àngale', '▁yi', '▁jot', 'ul', '▁woon', '▁a', '▁teg', '▁loxo', '▁ba', '▁ñu', '▁fa', '▁dàqee', '▁Alma', 'ŋ', '▁yi.', '▁Mu', 'y', '▁dëkk', '▁yu', '▁mel', '▁ni', '▁', ':', '▁Kan', 'tu', ',', '▁Abo', 'ŋ', ',', '▁Nko', 'm', ',', '▁Bu', 'm', ',', '▁Fu', 'm', 'baŋ', '▁', 'ak', '▁Baali', '.']
Wolof ids
[251, 72, 141, 13, 5129, 1187, 8, 31, 297, 81, 15, 129, 2346, 3010, 2319, 37, 7, 966, 995, 7, 56, 86, 865, 2663, 7, 1790, 1551, 176, 39, 341, 120, 7233, 524, 37, 512, 37, 331, 124, 55, 17, 216, 310, 29, 38, 50, 3166, 970, 222, 203, 251, 14, 176, 39, 112, 23, 8, 53, 979, 419, 7, 1611, 222, 7, 1399, 89, 7, 449, 89, 7, 1438, 89, 4973, 8, 31, 4224, 9]


#### Creating the T5 custom dataset

We have two possibilities to use the tokenizer for fine-tuning a T5 model. 

- We can use the `PreTrainedTokenizerFast` class for which we will provide the different special tokens.

In [125]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer1 = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
    padding_side="left",
)

- Or give directly the tokenizer to the `T5TokenizerFast` class.

In [126]:
from transformers import T5TokenizerFast

wrapped_tokenizer2 = T5TokenizerFast(
    tokenizer_object=tokenizer
)

Let us give them the sentences that we use as example. 

In [127]:
fr_encoding = wrapped_tokenizer1(french_sentence, max_length=30, padding='max_length', truncation=True)

fr_encoding

{'input_ids': [3, 3, 3, 3, 3, 3, 188, 441, 2158, 20, 925, 7, 329, 105, 40, 8, 738, 707, 206, 957, 7, 137, 127, 110, 3442, 309, 41, 4829, 2648, 9], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [128]:
wf_encoding = wrapped_tokenizer2(wolof_sentence, max_length=100, padding='max_length', truncation=True)

wf_encoding

{'input_ids': [251, 72, 141, 13, 5129, 1187, 8, 31, 297, 81, 15, 129, 2346, 3010, 2319, 37, 7, 966, 995, 7, 56, 86, 865, 2663, 7, 1790, 1551, 176, 39, 341, 120, 7233, 524, 37, 512, 37, 331, 124, 55, 17, 216, 310, 29, 38, 50, 3166, 970, 222, 203, 251, 14, 176, 39, 112, 23, 8, 53, 979, 419, 7, 1611, 222, 7, 1399, 89, 7, 449, 89, 7, 1438, 89, 4973, 8, 31, 4224, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

Let us decode the wolof sentence.

In [129]:
wrapped_tokenizer1.decode(wf_encoding.input_ids, skip_special_tokens=True)

'Mu ngi dale digaloom ak Kamerun gi ci moomeelu Farañse yi, wuti bëj-gànnaar, ca cati Aadamawa, ëmbaale dëkk yu ndaw-ndawaan yi Àngale yi jotul woon a teg loxo ba ñu fa dàqee Almaŋ yi. Muy dëkk yu mel ni : Kantu, Aboŋ, Nkom, Bum, Fumbaŋ ak Baali.'

We can see that the `T5Tokenizer` add padding to the right side of the sequence while the `PretrainedTokenizer` add the padding to the left side. We can change the padding side from the settings. But, for the next steps, let us directly use the `T5Tokenizer`.

**Note that we can augment the sentences when generating them like we did when using the `GPT2 model`.** See the following notebook for discussion on the augmentation method that we will use [augmentation](text_augmentation.ipynb). And for a more clear explanation of the augmentation methods in NLP tasks and training look at the following article [augment_or_not](https://direct.mit.edu/coli/article/48/1/5/108844/To-Augment-or-Not-to-Augment-A-Comparative-Study).

Let us verify, before creating the custom dataset, the max length that we can get from the corpora' tokens without considering the augmentation.

In [130]:
max_len = 0

for sent in sentences['french_corpus'].to_list() + sentences['wolof_corpus'].to_list():
    
    len_ids = len(wrapped_tokenizer2(sent).input_ids)
    
    if len_ids > max_len:
        
        max_len = len_ids

        

In [131]:
# let us print the max lengths
max_len

279

We find a maximum length of **279** tokens. But considering the augmentation we can obtain more than 279 tokens because it will add modifications on the words and then it can recognize only parts of them and divide them in multiple other tokens. Let us add to the max length the fifth of it. 

In [132]:
max_len += max_len // 5

max_len

334

It is time to create our custom dataset.

Signature:
```python
class T5SentenceDataset(Dataset):

    def __init__(
        self,
        data_path: str, 
        tokenizer: PreTrainedTokenizerFast
        corpus_1: str = "french_corpus",
        corpus_2: str = "wolof_corpus",
        max_len: int = 334,
        cp1_truncation: bool = False,
        cp2_truncation: bool = False,
        file_sep: str = ",",
        cp1_transformer: Union[TransformerSequences, None] = None,
        cp2_transformer: Union[TransformerSequences, None] = None,
        **kwargs):

        pass
```

In [145]:
%%writefile wolof-translate/wolof_translate/data/dataset_v2.py
from wolof_translate.utils.sent_transformers import TransformerSequences
from transformers import PreTrainedTokenizerFast
from torch.utils.data import Dataset
from typing import *
import pandas as pd
import torch
import re

class T5SentenceDataset(Dataset):

    def __init__(
        self,
        data_path: str, 
        tokenizer: PreTrainedTokenizerFast,
        corpus_1: str = "french_corpus",
        corpus_2: str = "wolof_corpus",
        max_len: int = 334,
        truncation: bool = False,
        file_sep: str = ",",
        cp1_transformer: Union[TransformerSequences, None] = None,
        cp2_transformer: Union[TransformerSequences, None] = None,
        **kwargs):
        
        # let us recuperate the data frame
        self.__sentences = pd.read_csv(data_path, sep=file_sep, **kwargs)
        
        # let us recuperate the tokenizer
        self.tokenizer = tokenizer
        
        # recuperate the first corpus' sentences
        self.__sentences_1 = self.__sentences[corpus_1].to_list()
        
        # recuperate the second corpus' sentences
        self.__sentences_2 = self.__sentences[corpus_2].to_list()
        
        # recuperate the length
        self.__length = len(self.__sentences_1)
        
        # let us recuperate the max len
        self.max_len = max_len
        
        # let us recuperate the truncation argument
        self.truncation = truncation
        
        # let us initialize the transformer
        self.cp1_transformer = cp1_transformer
        
        self.cp2_transformer = cp2_transformer
        
    def __getitem__(self, index):
        """Recuperate ids and attention masks of sentences at index

        Args:
            index (int): The index of the sentences to recuperate

        Returns:
            tuple: The `sentence to translate' ids`, `the attention mask of the sentence to translate`
            `the labels' ids`
        """
        sentence_1 = self.__sentences_1[index]
        
        sentence_2 = self.__sentences_2[index]
        
        # apply transformers if necessary
        if not self.cp1_transformer is None:
            
            sentence_1 = self.cp1_transformer(sentence_1) 
        
        if not self.cp2_transformer is None:
            
            sentence_2 = self.cp2_transformer(sentence_2)
        
        # let us encode the sentences (we provide the second sentence as labels to the tokenizer)
        data = self.tokenizer(
            sentence_1,
            truncation=self.truncation,
            max_length=self.max_len, 
            padding='max_length', 
            return_tensors="pt",
            text_target=sentence_2)
            
        
        return data.input_ids.squeeze(0), data.attention_mask.squeeze(0), data.labels.squeeze(0)
        
    def __len__(self):
        
        return self.__length
    
    def decode(self, labels: torch.Tensor):
        
        if labels.ndim < 2:
            
            labels = labels.unsqueeze(0)

        sentences = self.tokenizer.batch_decode(labels, skip_special_tokens=True)

        return sentences

Overwriting wolof-translate/wolof_translate/data/dataset_v2.py


In [146]:
%run wolof-translate/wolof_translate/data/dataset_v2.py

Let us generate some data with their masks and decode the labels.

**Note that we will use, when training the `T5 model`, train and test sets and not directly the full dataset**

In [147]:
# Initialize our custom dataset
dataset = T5SentenceDataset("data/extractions/new_data/corpora_v3.csv", wrapped_tokenizer2)

In [148]:
input_ids, mask, labels = next(iter(DataLoader(dataset, 10))) # generate 10 sentences

Let us print the input ids.

In [149]:
input_ids

tensor([[ 782,  336, 2555,  ...,    3,    3,    3],
        [4700,  683,   44,  ...,    3,    3,    3],
        [ 153,  178,   85,  ...,    3,    3,    3],
        ...,
        [ 286,  647,   10,  ...,    3,    3,    3],
        [ 411,  104, 1090,  ...,    3,    3,    3],
        [ 184, 6877,    7,  ...,    3,    3,    3]])

In [150]:
mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [151]:
labels

tensor([[4157, 3715,   43,  ...,    3,    3,    3],
        [5189,  530,   78,  ...,    3,    3,    3],
        [ 224,  343,   72,  ...,    3,    3,    3],
        ...,
        [ 733, 5268,  159,  ...,    3,    3,    3],
        [ 979,   84, 1215,  ...,    3,    3,    3],
        [ 235,   15,  522,  ...,    3,    3,    3]])

Let us decode the labels.

In [152]:
dataset.decode(labels)

['Doomu-aadama bu, ne ci ndey ak baay nga jóge.',
 'Mënunu leen a baň a gërëm ak a bëgg, doonte sax mën nanoo am xel ňaar ci ňoom.',
 'Waaye ňu ngi fi, ak seen xar-kanam, seen taxawaay, seen defin ak seen jikko, seeni njuumte, seeni yaakaar, seen melokaanu loxook baaraami tànk, seen meloy bët ak karaw, seen waxin, seeni xalaat, amaana sax at ma ňuy nar a génne àddina. Loolu lépp, day àgg fu sore ci nun.',
 'Bi ma delloo dëkk ba ma juddoo, dama faa meloon ni gan. Du kenn ku ma fa xam, safatul dara ci man. Li nu jóge Afrig jur ci man tiis wu réy. Su ma yaboo sax ni mënuma woon a nangu ni maak samay way-jur dëkkëtuñu Afrig. Ca laa tàmbalee gént ni sama yaay nit ku ñuul la, di sàkkal it sama bopp cosaan lu bees.',
 'Àddinay dox ba Baay tollu ci noppalug liggéey, dellusi Tugal dëkk ak ňun. Ci la ma leere ni moom moomoo doon doomu Afrig.',
 'Mu doon nag lu naqadee nangu.',
 'Damaa mujjoon a delloo sama xel démb ngir lijjanti lépp la ca léjoon.',
 'Kon fàttalikoo meññ téere bu ndaw bii.',
 'K